In [45]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
import pytz
import os
import patsy #for spline regression
import scipy #for non-negative least square 
import scipy as sp 
from scipy import stats
from scipy.optimize import nnls
from numpy.linalg import inv #for matrix and statistics
import scipy as sp
import math
import statsmodels.api as sm
from sklearn.cluster import AgglomerativeClustering
from sklearn.cluster import KMeans
import sklearn.cluster
import sklearn.linear_model
from sklearn import ensemble
from sklearn.utils import shuffle
from sklearn.metrics import mean_absolute_error
import sklearn.cluster
import sklearn.linear_model
from sklearn.svm import SVR
%matplotlib inline

In [46]:
#specify data source 
[region,zipcode] = ['PJM','08641']
#[region,zipcode]  = ['CAISO','92562']

In [47]:
mainDir = 'C:\Users\Admin\Dropbox\Active\EnergyProject\Thesis'
dataDir = mainDir + '/data/load/' + region +'/'

In [48]:
if region == 'CAISO':
    
    #timezone for local time adjustment
    localtz = 'US/Pacific'

    #specific to CAISO data
    TacName = 'Caiso_Totals'#'TAC_NORTH'
    
    datasource = 'raw/caiso_load_data_2013'
    
    d01 = pd.read_csv(dataDir + datasource + '/'+ '01.csv')
    d01 = d01[d01['TAC_ZONE_NAME']==TacName].sort(columns='INTERVALSTARTTIME_GMT')
    d02 = pd.read_csv(dataDir + datasource +'/'+ '02.csv')
    d02 = d02[d02['TAC_ZONE_NAME']==TacName].sort(columns='INTERVALSTARTTIME_GMT')
    d03 = pd.read_csv(dataDir + datasource +'/'+ '03.csv')
    d03 = d03[d03['TAC_ZONE_NAME']==TacName].sort(columns='INTERVALSTARTTIME_GMT')
    d04 = pd.read_csv(dataDir + datasource + '/'+ '04.csv')
    d04 = d04[d04['TAC_ZONE_NAME']==TacName].sort(columns='INTERVALSTARTTIME_GMT')
    d05 = pd.read_csv(dataDir + datasource + '/'+ '05.csv')
    d05 = d05[d05['TAC_ZONE_NAME']==TacName].sort(columns='INTERVALSTARTTIME_GMT')
    d06 = pd.read_csv(dataDir + datasource + '/'+ '06.csv')
    d06 = d06[d06['TAC_ZONE_NAME']==TacName].sort(columns='INTERVALSTARTTIME_GMT')
    d07 = pd.read_csv(dataDir + datasource + '/'+ '07.csv')
    d07 = d07[d07['TAC_ZONE_NAME']==TacName].sort(columns='INTERVALSTARTTIME_GMT')
    d08 = pd.read_csv(dataDir + datasource + '/'+ '08.csv')
    d08 = d08[d08['TAC_ZONE_NAME']==TacName].sort(columns='INTERVALSTARTTIME_GMT')
    d09 = pd.read_csv(dataDir + datasource + '/'+ '09.csv')
    d09 = d09[d09['TAC_ZONE_NAME']==TacName].sort(columns='INTERVALSTARTTIME_GMT')
    d10 = pd.read_csv(dataDir + datasource + '/'+ '10.csv')
    d10 = d10[d10['TAC_ZONE_NAME']==TacName].sort(columns='INTERVALSTARTTIME_GMT')
    d11 = pd.read_csv(dataDir + datasource + '/'+ '11.csv')
    d11 = d11[d11['TAC_ZONE_NAME']==TacName].sort(columns='INTERVALSTARTTIME_GMT')
    d12 = pd.read_csv(dataDir + datasource + '/'+ '12.csv')
    d12 = d12[d12['TAC_ZONE_NAME']==TacName].sort(columns='INTERVALSTARTTIME_GMT')
    data = pd.concat([d01,d02,d03,d04,d05,d06,d07,d08,d09,d10,d11,d12])
    
    datasource = 'raw/caiso_load_data_2014'
    
    d01 = pd.read_csv(dataDir + datasource + '/'+ '01.csv')
    d01 = d01[d01['TAC_ZONE_NAME']==TacName].sort(columns='INTERVALSTARTTIME_GMT')
    d02 = pd.read_csv(dataDir + datasource + '/'+ '02.csv')
    d02 = d02[d02['TAC_ZONE_NAME']==TacName].sort(columns='INTERVALSTARTTIME_GMT')
    d03 = pd.read_csv(dataDir + datasource + '/'+ '03.csv')
    d03 = d03[d03['TAC_ZONE_NAME']==TacName].sort(columns='INTERVALSTARTTIME_GMT')
    d04 = pd.read_csv(dataDir + datasource + '/'+ '04.csv')
    d04 = d04[d04['TAC_ZONE_NAME']==TacName].sort(columns='INTERVALSTARTTIME_GMT')
    d05 = pd.read_csv(dataDir + datasource + '/'+ '05.csv')
    d05 = d05[d05['TAC_ZONE_NAME']==TacName].sort(columns='INTERVALSTARTTIME_GMT')
    d06 = pd.read_csv(dataDir + datasource + '/'+ '06.csv')
    d06 = d06[d06['TAC_ZONE_NAME']==TacName].sort(columns='INTERVALSTARTTIME_GMT')
    d07 = pd.read_csv(dataDir + datasource + '/'+ '07.csv')
    d07 = d07[d07['TAC_ZONE_NAME']==TacName].sort(columns='INTERVALSTARTTIME_GMT')
    d08 = pd.read_csv(dataDir + datasource + '/'+ '08.csv')
    d08 = d08[d08['TAC_ZONE_NAME']==TacName].sort(columns='INTERVALSTARTTIME_GMT')
    d09 = pd.read_csv(dataDir + datasource + '/'+ '09.csv')
    d09 = d09[d09['TAC_ZONE_NAME']==TacName].sort(columns='INTERVALSTARTTIME_GMT')
    d10 = pd.read_csv(dataDir + datasource + '/'+ '10.csv')
    d10 = d10[d10['TAC_ZONE_NAME']==TacName].sort(columns='INTERVALSTARTTIME_GMT')
    d11 = pd.read_csv(dataDir + datasource + '/'+ '11.csv')
    d11 = d11[d11['TAC_ZONE_NAME']==TacName].sort(columns='INTERVALSTARTTIME_GMT')
    d12 = pd.read_csv(dataDir + datasource + '/'+ '12.csv')
    d12 = d12[d12['TAC_ZONE_NAME']==TacName].sort(columns='INTERVALSTARTTIME_GMT')
    data = pd.concat([data,d01,d02,d03,d04,d05,d06,d07,d08,d09,d10,d11,d12])

    data['tsLocal'] = data['INTERVALSTARTTIME_GMT'].map(lambda x: datetime.datetime.strptime(x[0:19],"%Y-%m-%dT%H:%M:%S").replace(tzinfo=pytz.utc)\
                                .astimezone(pytz.timezone(localtz)).strftime("%Y-%m-%d %H:%M:%S"))
    data['tsLocal'] = data['tsLocal'].map(lambda x: datetime.datetime.strptime(x,"%Y-%m-%d %H:%M:%S"))

    data = data[['tsLocal','MW']].reset_index().drop(['index'],1)
    data.columns = ['tsLocal','load']
    #delete double readings (can happen due to daylight savings)
    data = data.groupby(['tsLocal']).mean().reset_index()
    

In [49]:
if region == 'PJM':
    
    datasource = 'raw'
    #filename = 'pjm_e_load_data_2012.csv'
    filename = '2013-PJM-hourly-loads.xls'
    data = pd.read_excel(dataDir + datasource + '/'+filename,sheetname='RTO')
    filename = '2014-PJM-hourly-loads.xls'
    d = pd.read_excel(dataDir + datasource + '/'+filename,sheetname='RTO')
    data = pd.concat([data,d])
    
    data = data.loc[:,data.columns[0:26]]
    #set DAY as index
    data = data.set_index('DATE')
    #remove unnecessary column 'peak'
    data = data.drop('COMP', 1)
    #unstack data and rename columns
    data = data.unstack().reset_index()
    data.columns = ['hour','date','load']

    #convert hour and date to timestamp then sort by tsLocal
    data['tsLocal'] = data['date'].map(lambda x: datetime.datetime.strptime(x,"%m/%d/%Y"))+\
                    data['hour'].map(lambda x: datetime.timedelta(hours=int(str(x)[2:4])))
    data = data.sort(['tsLocal'])
    #don't need weekday for now
    #data['weekday'] = data['tsLocal'].map(lambda x: x.weekday())
    #reset index and drop uncesseary index
    data = data.reset_index()
    data = data.drop(['index','hour','date'], 1)

    #delete double readings (can happen due to daylight savings)
    data = data.groupby(['tsLocal']).mean().reset_index()

WARNING *** file size (6815616) not 512 + multiple of sector size (512)


In [50]:
#weekday info
data['weekday'] = data['tsLocal'].map(lambda x: x.weekday())
#date and hour info
#add date and hour, normalized to 1
data['d'] = data.tsLocal.map(lambda x: x.timetuple().tm_yday)/365.0
data['h'] = data.tsLocal.map(lambda x: x.timetuple().tm_hour)/24.0
loaddata = data

## Weather data

In [51]:
#mainDir = 'C:/Users/umnouyp/Dropbox/Active/EnergyProject/Thesis/PVreadingsStudies/main'
#mainDir = 'C:/Users/Tee/Dropbox/Active/EnergyProject/Thesis/PVreadingsStudies/main'
mainDir = 'C:\Users\Admin\Dropbox\Active\EnergyProject\Thesis'
#we can choose which data to look at.
dataDir = mainDir + '/data/solar/' + datasource +'/' + zipcode +'/'
outputDir = mainDir + '/output/'+ datasource +'/' + zipcode +'/'

In [52]:
weatherdatasource = '/data/weather/weathersource/hourly/'
if (zipcode == '08641')|(zipcode =='08640'):
    weatherdata = pd.read_csv(mainDir+weatherdatasource+'08641_20132014.csv')
if (zipcode == '92562')|(zipcode =='92563'):
    weatherdata = pd.read_csv(mainDir+weatherdatasource+'92563_20132014.csv')

weatherdata['tsLocal'] = weatherdata.timestamp.map(lambda x: x[0:10] + " " + x[11:19])
#weatherdata.tsLocal = weatherdata.tsLocal.map(lambda x: datetime.datetime.strptime(x,"%Y-%m-%d %H:%M:%S"))
if type(weatherdata.tsLocal[0])==str:
    weatherdata['tsLocal'] = weatherdata['tsLocal'].map(lambda x: datetime.datetime.strptime(x,"%Y-%m-%d %H:%M:%S"))
weatherdata.drop(['timestamp','country','postal_code'], axis=1, inplace=True)

In [53]:
if weatherdata.shape[0]!=len(set(weatherdata['tsLocal'])):
    for i in range(weatherdata.shape[0]):
        if weatherdata.loc[i,'tsLocal'] == weatherdata.loc[(i+1),'tsLocal']:
            print weatherdata.loc[i,'tsLocal']

# Basic time series construction

In [54]:
#specify scope
StartScopeDate = '2013-01-01'
EndScopeDate = '2015-01-01'
StartScopeTime =  datetime.datetime.strptime(StartScopeDate,"%Y-%m-%d")
EndScopeTime =  datetime.datetime.strptime(EndScopeDate,"%Y-%m-%d")

#Generate data frame from start to end for time series
tslist = []
nxt = StartScopeTime
while nxt < EndScopeTime:
    tslist.append(nxt)
    nxt += datetime.timedelta(minutes=60) #hour data now
        
Scope = pd.DataFrame(0, index = np.arange(len(tslist)), columns = ['tsLocal'])
Scope['tsLocal'] = tslist

#Now we can merge using Scope Table as a backbone of data structure.
d = pd.merge(Scope,weatherdata,on=['tsLocal'], how = 'left') 



In [55]:
#there is no need to do interpolation

#Now we can merge our backbone + weather with solar data
d = pd.merge(d,loaddata,on=['tsLocal'], how = 'left') 

In [56]:
#weather
d['cldCvr-1'] = float('NaN')
d['dewPt-1'] = float('NaN')
d['feelsLike-1'] = float('NaN')
d['precip-1'] = float('NaN')
d['relHum-1'] = float('NaN')
d['sfcPres-1'] = float('NaN')
d['snowfall-1'] = float('NaN')
d['spcHum-1'] = float('NaN')
d['temp-1'] = float('NaN')
d['windSpd-1'] = float('NaN')
d['wetBulb-1'] = float('NaN')
d.loc[1:d.shape[0],'cldCvr-1'] = list(d['cldCvr'].iloc[0:(d.shape[0]-1)])
d.loc[1:d.shape[0],'dewPt-1'] = list(d['dewPt'].iloc[0:(d.shape[0]-1)])
d.loc[1:d.shape[0],'feelsLike-1'] = list(d['feelsLike'].iloc[0:(d.shape[0]-1)])
d.loc[1:d.shape[0],'precip-1'] = list(d['precip'].iloc[0:(d.shape[0]-1)])
d.loc[1:d.shape[0],'relHum-1'] = list(d['relHum'].iloc[0:(d.shape[0]-1)])
d.loc[1:d.shape[0],'sfcPres-1'] = list(d['sfcPres'].iloc[0:(d.shape[0]-1)])
d.loc[1:d.shape[0],'snowfall-1'] = list(d['snowfall'].iloc[0:(d.shape[0]-1)])
d.loc[1:d.shape[0],'spcHum-1'] = list(d['spcHum'].iloc[0:(d.shape[0]-1)])
d.loc[1:d.shape[0],'temp-1'] = list(d['temp'].iloc[0:(d.shape[0]-1)])
d.loc[1:d.shape[0],'windSpd-1'] = list(d['windSpd'].iloc[0:(d.shape[0]-1)])
d.loc[1:d.shape[0],'wetBulb-1'] = list(d['wetBulb'].iloc[0:(d.shape[0]-1)])

#normalize weather value
d['cldCvr-1'] = (d['cldCvr-1']-np.min(d['cldCvr-1']))/(np.max(d['cldCvr-1'])-np.min(d['cldCvr-1']))
d['dewPt-1'] = (d['dewPt-1']-np.min(d['dewPt-1']))/(np.max(d['dewPt-1'])-np.min(d['dewPt-1']))
d['feelsLike-1'] = (d['feelsLike-1']-np.min(d['feelsLike-1']))/(np.max(d['feelsLike-1'])-np.min(d['feelsLike-1']))
d['precip-1'] = (d['precip-1']-np.min(d['precip-1']))/(np.max(d['precip-1'])-np.min(d['precip-1']))
d['relHum-1'] = (d['relHum-1']-np.min(d['relHum-1']))/(np.max(d['relHum-1'])-np.min(d['relHum-1']))
d['sfcPres-1'] = (d['sfcPres-1']-np.min(d['sfcPres-1']))/(np.max(d['sfcPres-1'])-np.min(d['sfcPres-1']))
d['snowfall-1'] = (d['snowfall-1']-np.min(d['snowfall-1']))/(np.max(d['snowfall-1'])-np.min(d['snowfall-1']))
d['spcHum-1'] = (d['spcHum-1']-np.min(d['spcHum-1']))/(np.max(d['spcHum-1'])-np.min(d['spcHum-1']))
d['temp-1'] = (d['temp-1']-np.min(d['temp-1']))/(np.max(d['temp-1'])-np.min(d['temp-1']))
d['windSpd-1'] = (d['windSpd-1']-np.min(d['windSpd-1']))/(np.max(d['windSpd-1'])-np.min(d['windSpd-1']))
d['wetBulb-1'] = (d['wetBulb-1']-np.min(d['wetBulb-1']))/(np.max(d['wetBulb-1'])-np.min(d['wetBulb-1']))

In [57]:
#timeseries of previous timestamps
d['load-1d'] = float('NaN') #previous day
d['load-2d'] = float('NaN') #previous day
d['load-3d'] = float('NaN') #previous day
d['load-4d'] = float('NaN') #previous day
d['load-5d'] = float('NaN') #previous day
d['load-6d'] = float('NaN') #previous day
d['load-1w'] = float('NaN') #previous 7 days
d['load-1h'] = float('NaN') #previous hr
d['load-1d-1h'] = float('NaN')
d['load-1w-1h'] = float('NaN')
d['load-2h'] = float('NaN') #previous hr
d['load-1d-2h'] = float('NaN') #previous hr
d['load-1w-2h'] = float('NaN') #previous hr

#24 hour shift = 24 shift in index.

d.loc[24:d.shape[0],'load-1d'] = list(d['load'].iloc[0:(d.shape[0]-24)])
d.loc[(24*2):d.shape[0],'load-2d'] = list(d['load'].iloc[0:(d.shape[0]-(24*2))])
d.loc[(24*3):d.shape[0],'load-3d'] = list(d['load'].iloc[0:(d.shape[0]-(24*3))])
d.loc[(24*4):d.shape[0],'load-4d'] = list(d['load'].iloc[0:(d.shape[0]-(24*4))])
d.loc[(24*5):d.shape[0],'load-5d'] = list(d['load'].iloc[0:(d.shape[0]-(24*5))])
d.loc[(24*6):d.shape[0],'load-6d'] = list(d['load'].iloc[0:(d.shape[0]-(24*6))])
d.loc[(24*7):d.shape[0],'load-1w'] = list(d['load'].iloc[0:(d.shape[0]-(24*7))])

d.loc[1:d.shape[0],'load-1h'] = list(d['load'].iloc[0:(d.shape[0]-1)])
d.loc[25:d.shape[0],'load-1d-1h'] = list(d['load'].iloc[0:(d.shape[0]-25)])
d.loc[(24*7+1):d.shape[0],'load-1w-1h'] = list(d['load'].iloc[0:(d.shape[0]-(24*7+1))])
d.loc[2:d.shape[0],'load-2h'] = list(d['load'].iloc[0:(d.shape[0]-2)])
d.loc[26:d.shape[0],'load-1d-2h'] = list(d['load'].iloc[0:(d.shape[0]-26)])
d.loc[(24*7+2):d.shape[0],'load-1w-2h'] = list(d['load'].iloc[0:(d.shape[0]-(24*7+2))])


In [58]:
d[d['tsLocal'].map(lambda x: x.year==2013)].shape,d[d['tsLocal'].map(lambda x: x.year==2014)].shape

((8760, 40), (8760, 40))

In [59]:
#adjust date correctly
datetime.date(2013,1,1).weekday(),datetime.date(2014,1,1).weekday()

(1, 2)

In [60]:
d[d.tsLocal.map(lambda x: x.date() == datetime.date(2014,1,1))][:1]

,tsLocal,cldCvr,dewPt,feelsLike,precip,relHum,sfcPres,snowfall,spcHum,temp,...,load-4d,load-5d,load-6d,load-1w,load-1h,load-1d-1h,load-1w-1h,load-2h,load-1d-2h,load-1w-2h
8760,2014-01-01,15,12.5,19.3,0,56.2,1022.8,0,1.64,26,...,90592.079,94092.666,90176.256,96348.839,97502.767,99773.947,99644.327,100683.011,104914.285,101478.199


In [61]:
d['load-1y'] = float('NaN')
#do one year shift. With weekday correction
d.loc[(8759):d.shape[0],'load-1y'] = list(d['load'].iloc[24:(d.shape[0]-(8759-24))])

Conclusion: to make a reference, use previous day first. For Monday and Saturday, use average of previous week and previous year with week alignment.

In [62]:
d['load_ref'] = d['load-1d']*((d['weekday']!=0)&(d['weekday']!=5))+\
                    d['load-1w']*(((d['weekday']==0)|(d['weekday']==5)))
                    #0.5*(d['load-1w']+d['load-1y'])*(((d['weekday']==0)|(d['weekday']==5)))

Add extra modification to make it smoother.

In [63]:
#ref... not use anymore 
#d['load_ref+1'] = float('NaN')
#d['load_ref-1'] = float('NaN')
#d.loc[1:d.shape[0],'load_ref-1'] = list(d['load_ref'].iloc[0:(d.shape[0]-1)])
#d.loc[0:(d.shape[0]-2),'load_ref+1'] = list(d['load_ref'].iloc[1:(d.shape[0])])
#d['load_ref'] = (d['load_ref']+d['load_ref-1']+d['load_ref+1'])/3.0

#for day
d['load-1d+1'] = float('NaN')
d['load-1d-1'] = float('NaN')
d.loc[1:d.shape[0],'load-1d-1'] = list(d['load-1d'].iloc[0:(d.shape[0]-1)])
d.loc[0:(d.shape[0]-2),'load-1d+1'] = list(d['load-1d'].iloc[1:(d.shape[0])])
d['load-1d_smooth'] = (d['load-1d']+d['load-1d-1']+d['load-1d+1'])/3.0

#for week
d['load-1w+1'] = float('NaN')
d['load-1w-1'] = float('NaN')
d.loc[1:d.shape[0],'load-1w-1'] = list(d['load-1w'].iloc[0:(d.shape[0]-1)])
d.loc[0:(d.shape[0]-2),'load-1w+1'] = list(d['load-1w'].iloc[1:(d.shape[0])])
d['load-1w_smooth'] = (d['load-1w']+d['load-1w-1']+d['load-1w+1'])/3.0

'''
#Define ratio r
d['r'] = d['load']/d['load_ref']
d['r-1'] = float('NaN')
d['r-2'] = float('NaN')
d['r-3'] = float('NaN')
d['r-4'] = float('NaN')
d['r-5'] = float('NaN')
d['r-6'] = float('NaN')
d.loc[1:d.shape[0],'r-1'] = list(d['r'].iloc[0:(d.shape[0]-1)])
d.loc[2:d.shape[0],'r-2'] = list(d['r'].iloc[0:(d.shape[0]-2)])
d.loc[3:d.shape[0],'r-3'] = list(d['r'].iloc[0:(d.shape[0]-3)])
d.loc[4:d.shape[0],'r-4'] = list(d['r'].iloc[0:(d.shape[0]-4)])
d.loc[5:d.shape[0],'r-5'] = list(d['r'].iloc[0:(d.shape[0]-5)])
d.loc[6:d.shape[0],'r-6'] = list(d['r'].iloc[0:(d.shape[0]-6)])
'''

#Define finer ratio
d['rd'] = d['load']/d['load-1d']
d['rw'] = d['load']/d['load-1w']
#d['ry'] = d['load']/d['load-1y']
d['rd-1'] = float('NaN')
d['rw-1'] = float('NaN')
#d['ry-1'] = float('NaN')
d['rd-2'] = float('NaN')
d['rw-2'] = float('NaN')
#d['ry-2'] = float('NaN')
d['rd-3'] = float('NaN')
d['rw-3'] = float('NaN')
#d['ry-2'] = float('NaN')
d.loc[1:d.shape[0],'rd-1'] = list(d['rd'].iloc[0:(d.shape[0]-1)])
d.loc[1:d.shape[0],'rw-1'] = list(d['rw'].iloc[0:(d.shape[0]-1)])
#d.loc[1:d.shape[0],'ry-1'] = list(d['ry'].iloc[0:(d.shape[0]-1)])
d.loc[2:d.shape[0],'rd-2'] = list(d['rd'].iloc[0:(d.shape[0]-2)])
d.loc[2:d.shape[0],'rw-2'] = list(d['rw'].iloc[0:(d.shape[0]-2)])
#d.loc[2:d.shape[0],'ry-2'] = list(d['ry'].iloc[0:(d.shape[0]-2)])
d.loc[3:d.shape[0],'rd-3'] = list(d['rd'].iloc[0:(d.shape[0]-3)])
d.loc[3:d.shape[0],'rw-3'] = list(d['rw'].iloc[0:(d.shape[0]-3)])
#d.loc[3:d.shape[0],'ry-3'] = list(d['ry'].iloc[0:(d.shape[0]-3)])

#feed this back to data
Data = d




#1-hr rolling prediction: Naive model, load and 'r' investigation

In [64]:
d = Data
#Truly random
#l = np.random.permutation(d.shape[0])
#d = d.iloc[l].reset_index(drop=True)
#offset = int(d.shape[0] * 0.8)

#Assign specific period
StartDate = datetime.date(2014,10,1)
offset = d[d.tsLocal.map(lambda x: x.date()) <StartDate].shape[0]

Data_train = d[:offset]
Data_test = d[offset:]

In [65]:
np.max(Data_test['load']), np.mean(Data_test['load'])

(120847.897, 86464.576358858176)

## Model Reference: Persistence

In [66]:
d_train = Data_train
d_test = Data_test
feature_names = ['rd-1','rw-1']
d_train = Data_train.loc[:,['tsLocal','load','load-1w','load-1d']+feature_names].dropna()
d_test = Data_test.loc[:,['tsLocal','load','load-1w','load-1d']+feature_names].dropna()

# run individual models for each day of week
d_test['load_predict']= float('NaN')

In [67]:
# with reference with previous day + week
d_test['load_predict']=float('NaN')
d_test['load_predict']=0.5*(d_test['rd-1']*d_test['load-1d']+d_test['rw-1']*d_test['load-1w'])

l_0 = d_test['load']-d_test['load_predict']
[mean_persistence,max_persistence] = [np.mean(np.absolute(l_0)),np.max(np.absolute(l_0))]

print [mean_persistence,max_persistence]

[606.91442847187125, 5133.1581509796379]


## Simple model: Auto-regression 1

In [68]:
feature_names = ['rw-1','rd-1']

d_train = Data_train.loc[:,['tsLocal','load','load-1w','load-1d','rw','rd']+feature_names].dropna()
d_test = Data_test.loc[:,['tsLocal','load','load-1w','load-1d','rw','rd']+feature_names].dropna()

d_test['load_predict']= float('NaN')

feature_names = ['rw-1']
X_train, y_train = d_train.loc[:,feature_names], d_train.loc[:,'rw']
X_test, y_test = d_test.loc[:,feature_names], d_test.loc[:,'rw']
mod = sklearn.linear_model.LinearRegression().fit(X_train,y_train)
d_test.loc[:,'load_predict_w'] = (mod.predict(X_test))*d_test.loc[:,'load-1w']

feature_names = ['rd-1']
X_train, y_train = d_train.loc[:,feature_names], d_train.loc[:,'rd']
X_test, y_test = d_test.loc[:,feature_names], d_test.loc[:,'rd']
mod = sklearn.linear_model.LinearRegression().fit(X_train,y_train)
d_test.loc[:,'load_predict_d'] = (mod.predict(X_test))*d_test.loc[:,'load-1d']


l_1 = d_test['load']-0.5*(d_test['load_predict_d']+d_test['load_predict_w'])
[mean_AR1,max_AR1] = [np.mean(np.absolute(l_1)),np.max(np.absolute(l_1))]

print [mean_AR1,max_AR1]

[608.55703863602821, 5132.273939469349]


In [69]:
feature_names = ['rw-1','rd-1']

d_train = Data_train.loc[:,['tsLocal','load','load-1w','load-1d','rw','rd']+feature_names].dropna()
d_test = Data_test.loc[:,['tsLocal','load','load-1w','load-1d','rw','rd']+feature_names].dropna()

# run individual models for each day of week
d_test['load_predict']= float('NaN')

for i in range(0,7):
    Index_train = d_train[d_train['tsLocal'].map(lambda x: ((x.weekday()==i)))].index
    Index_test = d_test[d_test['tsLocal'].map(lambda x: ((x.weekday()==i)))].index
    
    feature_names = ['rw-1']
    X_train, y_train = d_train.loc[Index_train,feature_names], d_train.loc[Index_train,'rw']
    X_test, y_test = d_test.loc[Index_test,feature_names], d_test.loc[Index_test,'rw']
    mod = sklearn.linear_model.LinearRegression().fit(X_train,y_train)
    d_test.loc[Index_test,'load_predict_w'] = (mod.predict(X_test))*d_test.loc[Index_test,'load-1w']
    
    feature_names = ['rd-1']
    X_train, y_train = d_train.loc[Index_train,feature_names], d_train.loc[Index_train,'rd']
    X_test, y_test = d_test.loc[Index_test,feature_names], d_test.loc[Index_test,'rd']
    mod = sklearn.linear_model.LinearRegression().fit(X_train,y_train)
    d_test.loc[Index_test,'load_predict_d'] = (mod.predict(X_test))*d_test.loc[Index_test,'load-1d']


l_1s = d_test['load']-0.5*(d_test['load_predict_d']+d_test['load_predict_w'])
[mean_AR1,max_AR1] = [np.mean(np.absolute(l_1s)),np.max(np.absolute(l_1s))]

print [mean_AR1,max_AR1]

[605.60835578653348, 5039.5821681385423]


## Simple model: Auto-regression 2

In [70]:
feature_names = ['rw-1','rd-1','rw-2','rd-2']

d_train = Data_train.loc[:,['tsLocal','load','load-1w','load-1d','rw','rd']+feature_names].dropna()
d_test = Data_test.loc[:,['tsLocal','load','load-1w','load-1d','rw','rd']+feature_names].dropna()

d_test['load_predict']= float('NaN')

feature_names = ['rw-1','rw-2']
X_train, y_train = d_train.loc[:,feature_names], d_train.loc[:,'rw']
X_test, y_test = d_test.loc[:,feature_names], d_test.loc[:,'rw']
mod = sklearn.linear_model.LinearRegression().fit(X_train,y_train)
d_test.loc[:,'load_predict_w'] = (mod.predict(X_test))*d_test.loc[:,'load-1w']

feature_names = ['rd-1','rd-2']
X_train, y_train = d_train.loc[:,feature_names], d_train.loc[:,'rd']
X_test, y_test = d_test.loc[:,feature_names], d_test.loc[:,'rd']
mod = sklearn.linear_model.LinearRegression().fit(X_train,y_train)
d_test.loc[:,'load_predict_d'] = (mod.predict(X_test))*d_test.loc[:,'load-1d']


l_2 = d_test['load']-0.5*(d_test['load_predict_d']+d_test['load_predict_w'])
[mean_AR2,max_AR2] = [np.mean(np.absolute(l_2)),np.max(np.absolute(l_2))]

print [mean_AR2,max_AR2]

[415.61181117048176, 5729.0693479674519]


In [71]:
feature_names = ['rw-1','rd-1','rw-2','rd-2']

d_train = Data_train.loc[:,['tsLocal','load','load-1w','load-1d','rw','rd']+feature_names].dropna()
d_test = Data_test.loc[:,['tsLocal','load','load-1w','load-1d','rw','rd']+feature_names].dropna()

# run individual models for each day of week
d_test['load_predict']= float('NaN')

for i in range(0,7):
    Index_train = d_train[d_train['tsLocal'].map(lambda x: ((x.weekday()==i)))].index
    Index_test = d_test[d_test['tsLocal'].map(lambda x: ((x.weekday()==i)))].index
    
    feature_names = ['rw-1','rw-2']
    X_train, y_train = d_train.loc[Index_train,feature_names], d_train.loc[Index_train,'rw']
    X_test, y_test = d_test.loc[Index_test,feature_names], d_test.loc[Index_test,'rw']
    mod = sklearn.linear_model.LinearRegression().fit(X_train,y_train)
    d_test.loc[Index_test,'load_predict_w'] = (mod.predict(X_test))*d_test.loc[Index_test,'load-1w']

    feature_names = ['rd-1','rd-2']
    X_train, y_train = d_train.loc[Index_train,feature_names], d_train.loc[Index_train,'rd']
    X_test, y_test = d_test.loc[Index_test,feature_names], d_test.loc[Index_test,'rd']
    mod = sklearn.linear_model.LinearRegression().fit(X_train,y_train)
    d_test.loc[Index_test,'load_predict_d'] = (mod.predict(X_test))*d_test.loc[Index_test,'load-1d']

l_2s = d_test['load']-0.5*(d_test['load_predict_d']+d_test['load_predict_w'])
[mean_AR2,max_AR2] = [np.mean(np.absolute(l_2s)),np.max(np.absolute(l_2s))]

print [mean_AR2,max_AR2]

[410.43136434896189, 5757.4896555972227]


## Simple model: Auto-regression 3

In [72]:
feature_names = ['rw-1','rd-1','rw-2','rd-2','rw-3','rd-3']

d_train = Data_train.loc[:,['tsLocal','load','load-1w','load-1d','rw','rd']+feature_names].dropna()
d_test = Data_test.loc[:,['tsLocal','load','load-1w','load-1d','rw','rd']+feature_names].dropna()

d_test['load_predict']= float('NaN')

feature_names = ['rw-1','rw-2','rw-3']
X_train, y_train = d_train.loc[:,feature_names], d_train.loc[:,'rw']
X_test, y_test = d_test.loc[:,feature_names], d_test.loc[:,'rw']
mod = sklearn.linear_model.LinearRegression().fit(X_train,y_train)
d_test.loc[:,'load_predict_w'] = (mod.predict(X_test))*d_test.loc[:,'load-1w']

feature_names = ['rd-1','rd-2','rd-3']
X_train, y_train = d_train.loc[:,feature_names], d_train.loc[:,'rd']
X_test, y_test = d_test.loc[:,feature_names], d_test.loc[:,'rd']
mod = sklearn.linear_model.LinearRegression().fit(X_train,y_train)
d_test.loc[:,'load_predict_d'] = (mod.predict(X_test))*d_test.loc[:,'load-1d']


l_3 = d_test['load']-0.5*(d_test['load_predict_d']+d_test['load_predict_w'])
[mean_AR2,max_AR2] = [np.mean(np.absolute(l_3)),np.max(np.absolute(l_3))]

print [mean_AR2,max_AR2]

[412.60752976039618, 6804.4477671432251]


In [73]:
feature_names = ['rw-1','rd-1','rw-2','rd-2','rw-3','rd-3']

d_train = Data_train.loc[:,['tsLocal','load','load-1w','load-1d','rw','rd']+feature_names].dropna()
d_test = Data_test.loc[:,['tsLocal','load','load-1w','load-1d','rw','rd']+feature_names].dropna()

# run individual models for each day of week
d_test['load_predict']= float('NaN')

for i in range(0,7):
    Index_train = d_train[d_train['tsLocal'].map(lambda x: ((x.weekday()==i)))].index
    Index_test = d_test[d_test['tsLocal'].map(lambda x: ((x.weekday()==i)))].index
    
    feature_names = ['rw-1','rw-2','rw-3']
    X_train, y_train = d_train.loc[Index_train,feature_names], d_train.loc[Index_train,'rw']
    X_test, y_test = d_test.loc[Index_test,feature_names], d_test.loc[Index_test,'rw']
    mod = sklearn.linear_model.LinearRegression().fit(X_train,y_train)
    d_test.loc[Index_test,'load_predict_w'] = (mod.predict(X_test))*d_test.loc[Index_test,'load-1w']

    feature_names = ['rd-1','rd-2','rd-3']
    X_train, y_train = d_train.loc[Index_train,feature_names], d_train.loc[Index_train,'rd']
    X_test, y_test = d_test.loc[Index_test,feature_names], d_test.loc[Index_test,'rd']
    mod = sklearn.linear_model.LinearRegression().fit(X_train,y_train)
    d_test.loc[Index_test,'load_predict_d'] = (mod.predict(X_test))*d_test.loc[Index_test,'load-1d']

l_3s = d_test['load']-0.5*(d_test['load_predict_d']+d_test['load_predict_w'])
[mean_AR2,max_AR2] = [np.mean(np.absolute(l_3s)),np.max(np.absolute(l_3s))]

print [mean_AR2,max_AR2]

[406.01116201828273, 6688.253955431428]


## Complex model: SVR

In [74]:
#Parameter for SVR
from sklearn.svm import SVR
C = 2.0
epsilon=0.0005

In [75]:
feature_names = ['rw-1','rd-1']

d_train = Data_train.loc[:,['tsLocal','load','load_ref','load-1w','load','rw','rd']+feature_names].dropna()
d_test = Data_test.loc[:,['tsLocal','load','load_ref','load-1w','load-1d','rw','rd']+feature_names].dropna()

# run individual models for each day of week
d_test['load_predict']= float('NaN')

feature_names = ['rw-1']
X_train, y_train = d_train.loc[:,feature_names], d_train.loc[:,'rw']
X_test, y_test = d_test.loc[:,feature_names], d_test.loc[:,'rw']
mod = SVR(C=C, epsilon=epsilon).fit(X_train,y_train)
d_test.loc[:,'load_predict_w'] = (mod.predict(X_test))*d_test.loc[:,'load-1w']

feature_names = ['rd-1']
X_train, y_train = d_train.loc[:,feature_names], d_train.loc[:,'rd']
X_test, y_test = d_test.loc[:,feature_names], d_test.loc[:,'rd']
mod = SVR(C=C, epsilon=epsilon).fit(X_train,y_train)
d_test.loc[:,'load_predict_d'] = (mod.predict(X_test))*d_test.loc[:,'load-1d']
        
l = d_test['load']-0.5*(d_test['load_predict_d']+d_test['load_predict_w'])
[mean_set1,max_set1] = [np.mean(np.absolute(l)),np.max(np.absolute(l))]

print [mean_set1,max_set1]

[609.81693985082154, 5157.5662990037381]


Tuning

(C=1.0, epsilon=0.1) = [1480.2323518060184, 8559.8547304030071]

(C=1.0, epsilon=0.05) = [932.15703092060403, 7013.5933323490608]

(C=1.0, epsilon=0.01) = [915.29783292919194, 6971.3207346325507]

(C=1.0, epsilon=0.005) = [912.89754875693814, 6935.2838413140271]

(C=1.0, epsilon=0.001) = [912.369922908896, 6919.9601870007464]

(C=1.0, epsilon=0.0005) = [912.39092358324729, 6917.8635301878967]

(C=1.0, epsilon=0.0001) = [912.94493019560116, 6930.3295250666561]

(C=2.0, epsilon=0.0005) = [912.06752424337424, 6908.2205535543326]

(C=5.0, epsilon=0.0005) = [912.30769907567321, 6909.4569672284269]



In [76]:
feature_names = ['rw-1','rd-1','rw-2','rd-2']

d_train = Data_train.loc[:,['tsLocal','load','load_ref','load-1w','load','rw','rd']+feature_names].dropna()
d_test = Data_test.loc[:,['tsLocal','load','load_ref','load-1w','load-1d','rw','rd']+feature_names].dropna()

# run individual models for each day of week
d_test['load_predict']= float('NaN')

feature_names = ['rw-1','rw-2']
X_train, y_train = d_train.loc[:,feature_names], d_train.loc[:,'rw']
X_test, y_test = d_test.loc[:,feature_names], d_test.loc[:,'rw']
mod = SVR(C=C, epsilon=epsilon).fit(X_train,y_train)
d_test.loc[:,'load_predict_w'] = (mod.predict(X_test))*d_test.loc[:,'load-1w']

feature_names = ['rd-1','rd-2']
X_train, y_train = d_train.loc[:,feature_names], d_train.loc[:,'rd']
X_test, y_test = d_test.loc[:,feature_names], d_test.loc[:,'rd']
mod = SVR(C=C, epsilon=epsilon).fit(X_train,y_train)
d_test.loc[:,'load_predict_d'] = (mod.predict(X_test))*d_test.loc[:,'load-1d']
        
l = d_test['load']-0.5*(d_test['load_predict_d']+d_test['load_predict_w'])
[mean_set2,max_set2] = [np.mean(np.absolute(l)),np.max(np.absolute(l))]

print [mean_set2,max_set2]

[413.81659610998167, 5703.6231565542403]


In [77]:
feature_names = ['rw-1','rd-1','d','h']

d_train = Data_train.loc[:,['tsLocal','load','load_ref','load-1w','load','rw','rd']+feature_names].dropna()
d_test = Data_test.loc[:,['tsLocal','load','load_ref','load-1w','load-1d','rw','rd']+feature_names].dropna()

# run individual models for each day of week
d_test['load_predict']= float('NaN')

feature_names = ['rw-1','d','h']
X_train, y_train = d_train.loc[:,feature_names], d_train.loc[:,'rw']
X_test, y_test = d_test.loc[:,feature_names], d_test.loc[:,'rw']
mod = SVR(C=C, epsilon=epsilon).fit(X_train,y_train)
d_test.loc[:,'load_predict_w'] = (mod.predict(X_test))*d_test.loc[:,'load-1w']

feature_names = ['rd-1','d','h']
X_train, y_train = d_train.loc[:,feature_names], d_train.loc[:,'rd']
X_test, y_test = d_test.loc[:,feature_names], d_test.loc[:,'rd']
mod = SVR(C=C, epsilon=epsilon).fit(X_train,y_train)
d_test.loc[:,'load_predict_d'] = (mod.predict(X_test))*d_test.loc[:,'load-1d']
        
l = d_test['load']-0.5*(d_test['load_predict_d']+d_test['load_predict_w'])
[mean_set3,max_set3] = [np.mean(np.absolute(l)),np.max(np.absolute(l))]

print [mean_set3,max_set3]

[611.9804346901193, 5033.252334863646]


In [78]:
feature_names = ['rw-1','rd-1','rw-2','rd-2','d','h']

d_train = Data_train.loc[:,['tsLocal','load','load_ref','load-1w','load','rw','rd']+feature_names].dropna()
d_test = Data_test.loc[:,['tsLocal','load','load_ref','load-1w','load-1d','rw','rd']+feature_names].dropna()

# run individual models for each day of week
d_test['load_predict']= float('NaN')

feature_names = ['rw-1','rw-2','d','h']
X_train, y_train = d_train.loc[:,feature_names], d_train.loc[:,'rw']
X_test, y_test = d_test.loc[:,feature_names], d_test.loc[:,'rw']
mod = SVR(C=C, epsilon=epsilon).fit(X_train,y_train)
d_test.loc[:,'load_predict_w'] = (mod.predict(X_test))*d_test.loc[:,'load-1w']

feature_names = ['rd-1','rd-2','d','h']
X_train, y_train = d_train.loc[:,feature_names], d_train.loc[:,'rd']
X_test, y_test = d_test.loc[:,feature_names], d_test.loc[:,'rd']
mod = SVR(C=C, epsilon=epsilon).fit(X_train,y_train)
d_test.loc[:,'load_predict_d'] = (mod.predict(X_test))*d_test.loc[:,'load-1d']
        
l = d_test['load']-0.5*(d_test['load_predict_d']+d_test['load_predict_w'])
[mean_set4,max_set4] = [np.mean(np.absolute(l)),np.max(np.absolute(l))]

print [mean_set4,max_set4]

[411.82212750981085, 5673.055868859723]


In [79]:
feature_names = ['rw-1','rd-1','d','h']
feature_names = feature_names +['cldCvr-1','dewPt-1','feelsLike-1','precip-1','relHum-1','sfcPres-1','snowfall-1',
                        'spcHum-1','temp-1','windSpd-1','wetBulb-1']

d_train = Data_train.loc[:,['tsLocal','load','load_ref','load-1w','load','rw','rd']+feature_names].dropna()
d_test = Data_test.loc[:,['tsLocal','load','load_ref','load-1w','load-1d','rw','rd']+feature_names].dropna()

# run individual models for each day of week
d_test['load_predict']= float('NaN')

feature_names = ['rw-1','d','h']
feature_names = feature_names +['cldCvr-1','dewPt-1','feelsLike-1','precip-1','relHum-1','sfcPres-1','snowfall-1',
                        'spcHum-1','temp-1','windSpd-1','wetBulb-1']
X_train, y_train = d_train.loc[:,feature_names], d_train.loc[:,'rw']
X_test, y_test = d_test.loc[:,feature_names], d_test.loc[:,'rw']
mod = SVR(C=C, epsilon=epsilon).fit(X_train,y_train)
d_test.loc[:,'load_predict_w'] = (mod.predict(X_test))*d_test.loc[:,'load-1w']

feature_names = ['rd-1','d','h']
feature_names = feature_names +['cldCvr-1','dewPt-1','feelsLike-1','precip-1','relHum-1','sfcPres-1','snowfall-1',
                        'spcHum-1','temp-1','windSpd-1','wetBulb-1']
X_train, y_train = d_train.loc[:,feature_names], d_train.loc[:,'rd']
X_test, y_test = d_test.loc[:,feature_names], d_test.loc[:,'rd']
mod = SVR(C=C, epsilon=epsilon).fit(X_train,y_train)
d_test.loc[:,'load_predict_d'] = (mod.predict(X_test))*d_test.loc[:,'load-1d']
        
l = d_test['load']-0.5*(d_test['load_predict_d']+d_test['load_predict_w'])
[mean_set5,max_set5] = [np.mean(np.absolute(l)),np.max(np.absolute(l))]

print [mean_set5,max_set5]

[623.20768399029555, 6699.8668418547604]


In [80]:
feature_names = ['rw-1','rd-1','rw-2','rd-2','d','h']
feature_names = feature_names +['cldCvr-1','dewPt-1','feelsLike-1','precip-1','relHum-1','sfcPres-1','snowfall-1',
                        'spcHum-1','temp-1','windSpd-1','wetBulb-1']

d_train = Data_train.loc[:,['tsLocal','load','load_ref','load-1w','load','rw','rd']+feature_names].dropna()
d_test = Data_test.loc[:,['tsLocal','load','load_ref','load-1w','load-1d','rw','rd']+feature_names].dropna()

# run individual models for each day of week
d_test['load_predict']= float('NaN')

feature_names = ['rw-1','rw-2','d','h']
feature_names = feature_names +['cldCvr-1','dewPt-1','feelsLike-1','precip-1','relHum-1','sfcPres-1','snowfall-1',
                        'spcHum-1','temp-1','windSpd-1','wetBulb-1']
X_train, y_train = d_train.loc[:,feature_names], d_train.loc[:,'rw']
X_test, y_test = d_test.loc[:,feature_names], d_test.loc[:,'rw']
mod = SVR(C=C, epsilon=epsilon).fit(X_train,y_train)
d_test.loc[:,'load_predict_w'] = (mod.predict(X_test))*d_test.loc[:,'load-1w']

feature_names = ['rd-1','rd-2','d','h']
feature_names = feature_names +['cldCvr-1','dewPt-1','feelsLike-1','precip-1','relHum-1','sfcPres-1','snowfall-1',
                        'spcHum-1','temp-1','windSpd-1','wetBulb-1']
X_train, y_train = d_train.loc[:,feature_names], d_train.loc[:,'rd']
X_test, y_test = d_test.loc[:,feature_names], d_test.loc[:,'rd']
mod = SVR(C=C, epsilon=epsilon).fit(X_train,y_train)
d_test.loc[:,'load_predict_d'] = (mod.predict(X_test))*d_test.loc[:,'load-1d']
        
l = d_test['load']-0.5*(d_test['load_predict_d']+d_test['load_predict_w'])
[mean_set6,max_set6] = [np.mean(np.absolute(l)),np.max(np.absolute(l))]

print [mean_set6,max_set6]

[418.73167749911323, 5513.9618747321074]


## Complex model: GBR

In [81]:
#Gradient Boosting Regression
###############################################################################
# Fit regression model
params = {'n_estimators': 1000, 'max_depth': 4, 'min_samples_split': 2,
          'learning_rate': 0.1, 'loss': 'lad'}
mod = ensemble.GradientBoostingRegressor(**params)

In [82]:
feature_names = ['rw-1','rd-1']

d_train = Data_train.loc[:,['tsLocal','load','load_ref','load-1w','load','rw','rd']+feature_names].dropna()
d_test = Data_test.loc[:,['tsLocal','load','load_ref','load-1w','load-1d','rw','rd']+feature_names].dropna()

# run individual models for each day of week
d_test['load_predict']= float('NaN')

feature_names = ['rw-1']
X_train, y_train = d_train.loc[:,feature_names], d_train.loc[:,'rw']
X_test, y_test = d_test.loc[:,feature_names], d_test.loc[:,'rw']
mod.fit(X_train, y_train)
d_test.loc[:,'load_predict_w'] = (mod.predict(X_test))*d_test.loc[:,'load-1w']

feature_names = ['rd-1']
X_train, y_train = d_train.loc[:,feature_names], d_train.loc[:,'rd']
X_test, y_test = d_test.loc[:,feature_names], d_test.loc[:,'rd']
mod.fit(X_train, y_train)
d_test.loc[:,'load_predict_d'] = (mod.predict(X_test))*d_test.loc[:,'load-1d']
        
l = d_test['load']-0.5*(d_test['load_predict_d']+d_test['load_predict_w'])
[mean_set1,max_set1] = [np.mean(np.absolute(l)),np.max(np.absolute(l))]

print [mean_set1,max_set1]

[615.52967452746748, 5110.8343581610243]


In [83]:
feature_names = ['rw-1','rd-1','rw-2','rd-2']

d_train = Data_train.loc[:,['tsLocal','load','load_ref','load-1w','load','rw','rd']+feature_names].dropna()
d_test = Data_test.loc[:,['tsLocal','load','load_ref','load-1w','load-1d','rw','rd']+feature_names].dropna()

# run individual models for each day of week
d_test['load_predict']= float('NaN')

feature_names = ['rw-1','rw-2']
X_train, y_train = d_train.loc[:,feature_names], d_train.loc[:,'rw']
X_test, y_test = d_test.loc[:,feature_names], d_test.loc[:,'rw']
mod.fit(X_train, y_train)
d_test.loc[:,'load_predict_w'] = (mod.predict(X_test))*d_test.loc[:,'load-1w']

feature_names = ['rd-1','rd-2']
X_train, y_train = d_train.loc[:,feature_names], d_train.loc[:,'rd']
X_test, y_test = d_test.loc[:,feature_names], d_test.loc[:,'rd']
mod.fit(X_train, y_train)
d_test.loc[:,'load_predict_d'] = (mod.predict(X_test))*d_test.loc[:,'load-1d']
        
l = d_test['load']-0.5*(d_test['load_predict_d']+d_test['load_predict_w'])
[mean_set2,max_set2] = [np.mean(np.absolute(l)),np.max(np.absolute(l))]

print [mean_set2,max_set2]

[416.96735072715342, 4525.0665360255225]


In [84]:
feature_names = ['rw-1','rd-1','d','h']

d_train = Data_train.loc[:,['tsLocal','load','load_ref','load-1w','load','rw','rd']+feature_names].dropna()
d_test = Data_test.loc[:,['tsLocal','load','load_ref','load-1w','load-1d','rw','rd']+feature_names].dropna()

# run individual models for each day of week
d_test['load_predict']= float('NaN')

feature_names = ['rw-1','d','h']
X_train, y_train = d_train.loc[:,feature_names], d_train.loc[:,'rw']
X_test, y_test = d_test.loc[:,feature_names], d_test.loc[:,'rw']
mod.fit(X_train, y_train)
d_test.loc[:,'load_predict_w'] = (mod.predict(X_test))*d_test.loc[:,'load-1w']

feature_names = ['rd-1','d','h']
X_train, y_train = d_train.loc[:,feature_names], d_train.loc[:,'rd']
X_test, y_test = d_test.loc[:,feature_names], d_test.loc[:,'rd']
mod.fit(X_train, y_train)
d_test.loc[:,'load_predict_d'] = (mod.predict(X_test))*d_test.loc[:,'load-1d']
        
l = d_test['load']-0.5*(d_test['load_predict_d']+d_test['load_predict_w'])
[mean_set3,max_set3] = [np.mean(np.absolute(l)),np.max(np.absolute(l))]

print [mean_set3,max_set3]

[585.82806851205612, 5154.3960097080999]


In [85]:
feature_names = ['rw-1','rd-1','rw-2','rd-2','d','h']

d_train = Data_train.loc[:,['tsLocal','load','load_ref','load-1w','load','rw','rd']+feature_names].dropna()
d_test = Data_test.loc[:,['tsLocal','load','load_ref','load-1w','load-1d','rw','rd']+feature_names].dropna()

# run individual models for each day of week
d_test['load_predict']= float('NaN')

feature_names = ['rw-1','rw-2','d','h']
X_train, y_train = d_train.loc[:,feature_names], d_train.loc[:,'rw']
X_test, y_test = d_test.loc[:,feature_names], d_test.loc[:,'rw']
mod.fit(X_train, y_train)
d_test.loc[:,'load_predict_w'] = (mod.predict(X_test))*d_test.loc[:,'load-1w']

feature_names = ['rd-1','rd-2','d','h']
X_train, y_train = d_train.loc[:,feature_names], d_train.loc[:,'rd']
X_test, y_test = d_test.loc[:,feature_names], d_test.loc[:,'rd']
mod.fit(X_train, y_train)
d_test.loc[:,'load_predict_d'] = (mod.predict(X_test))*d_test.loc[:,'load-1d']
        
l = d_test['load']-0.5*(d_test['load_predict_d']+d_test['load_predict_w'])
[mean_set4,max_set4] = [np.mean(np.absolute(l)),np.max(np.absolute(l))]

print [mean_set4,max_set4]

[397.76351695116631, 4383.7841217015521]


In [86]:
feature_names = ['rw-1','rd-1','d','h']
feature_names = feature_names +['cldCvr-1','dewPt-1','feelsLike-1','precip-1','relHum-1','sfcPres-1','snowfall-1',
                        'spcHum-1','temp-1','windSpd-1','wetBulb-1']

d_train = Data_train.loc[:,['tsLocal','load','load_ref','load-1w','load','rw','rd']+feature_names].dropna()
d_test = Data_test.loc[:,['tsLocal','load','load_ref','load-1w','load-1d','rw','rd']+feature_names].dropna()

# run individual models for each day of week
d_test['load_predict']= float('NaN')

feature_names = ['rw-1','d','h']
feature_names = feature_names +['cldCvr-1','dewPt-1','feelsLike-1','precip-1','relHum-1','sfcPres-1','snowfall-1',
                        'spcHum-1','temp-1','windSpd-1','wetBulb-1']
X_train, y_train = d_train.loc[:,feature_names], d_train.loc[:,'rw']
X_test, y_test = d_test.loc[:,feature_names], d_test.loc[:,'rw']
mod.fit(X_train, y_train)
d_test.loc[:,'load_predict_w'] = (mod.predict(X_test))*d_test.loc[:,'load-1w']

feature_names = ['rd-1','d','h']
feature_names = feature_names +['cldCvr-1','dewPt-1','feelsLike-1','precip-1','relHum-1','sfcPres-1','snowfall-1',
                        'spcHum-1','temp-1','windSpd-1','wetBulb-1']
X_train, y_train = d_train.loc[:,feature_names], d_train.loc[:,'rd']
X_test, y_test = d_test.loc[:,feature_names], d_test.loc[:,'rd']
mod.fit(X_train, y_train)
d_test.loc[:,'load_predict_d'] = (mod.predict(X_test))*d_test.loc[:,'load-1d']
        
l = d_test['load']-0.5*(d_test['load_predict_d']+d_test['load_predict_w'])
[mean_set5,max_set5] = [np.mean(np.absolute(l)),np.max(np.absolute(l))]

print [mean_set5,max_set5]

[588.84203837056452, 5199.4528634950111]


In [87]:
feature_names = ['rw-1','rd-1','rw-2','rd-2','d','h']
feature_names = feature_names +['cldCvr-1','dewPt-1','feelsLike-1','precip-1','relHum-1','sfcPres-1','snowfall-1',
                        'spcHum-1','temp-1','windSpd-1','wetBulb-1']

d_train = Data_train.loc[:,['tsLocal','load','load_ref','load-1w','load','rw','rd']+feature_names].dropna()
d_test = Data_test.loc[:,['tsLocal','load','load_ref','load-1w','load-1d','rw','rd']+feature_names].dropna()

# run individual models for each day of week
d_test['load_predict']= float('NaN')

feature_names = ['rw-1','rw-2','d','h']
feature_names = feature_names +['cldCvr-1','dewPt-1','feelsLike-1','precip-1','relHum-1','sfcPres-1','snowfall-1',
                        'spcHum-1','temp-1','windSpd-1','wetBulb-1']
X_train, y_train = d_train.loc[:,feature_names], d_train.loc[:,'rw']
X_test, y_test = d_test.loc[:,feature_names], d_test.loc[:,'rw']
mod.fit(X_train, y_train)
d_test.loc[:,'load_predict_w'] = (mod.predict(X_test))*d_test.loc[:,'load-1w']

feature_names = ['rd-1','rd-2','d','h']
feature_names = feature_names +['cldCvr-1','dewPt-1','feelsLike-1','precip-1','relHum-1','sfcPres-1','snowfall-1',
                        'spcHum-1','temp-1','windSpd-1','wetBulb-1']
X_train, y_train = d_train.loc[:,feature_names], d_train.loc[:,'rd']
X_test, y_test = d_test.loc[:,feature_names], d_test.loc[:,'rd']
mod.fit(X_train, y_train)
d_test.loc[:,'load_predict_d'] = (mod.predict(X_test))*d_test.loc[:,'load-1d']
        
l = d_test['load']-0.5*(d_test['load_predict_d']+d_test['load_predict_w'])
[mean_set6,max_set6] = [np.mean(np.absolute(l)),np.max(np.absolute(l))]

print [mean_set6,max_set6]

[415.74295336753255, 4364.0688217284187]


Tuning

(n_estimators=1000, max_depth = 4, min_sample_split = 1, learning_rate = 0.01) = [718.14909441670454, 8112.0216666666674]

(n_estimators=1000, max_depth = 3, min_sample_split = 1, learning_rate = 0.01) = [718.14909441670454, 8112.0216666666674]

(n_estimators=1000, max_depth = 5, min_sample_split = 1, learning_rate = 0.01) = [718.14909441670454, 8112.0216666666674]

(n_estimators=500, max_depth = 4, min_sample_split = 1, learning_rate = 0.01) = [718.14909441670454, 8112.0216666666674]

(n_estimators=2000, max_depth = 4, min_sample_split = 1, learning_rate = 0.01) = [718.14909441670454, 8112.0216666666674]

(n_estimators=1000, max_depth = 4, min_sample_split = 2, learning_rate = 0.01) = [718.14909441670454, 8112.0216666666674]

(n_estimators=1000, max_depth = 4, min_sample_split = 1, learning_rate = 0.05) = [718.14909441670454, 8112.0216666666674]

(n_estimators=1000, max_depth = 4, min_sample_split = 1, learning_rate = 0.1) = [718.14909441670454, 8112.0216666666674]


So far, we conclude that SVR is a good model. However, the model has consistent error in a certain date and time. This is because even though we take different reference curves, different day of week seems to have effect. In addition, load reference with last year data is actually bad. So we change reference curve to only yesterday, last week, and treat each day of week individually. We also need to aware to ratio cross-over between days. The best way to do is to use rd and rw separately.

In [88]:
feature_names = ['rw-1','rd-1','rw-2','rd-2','d','h']

d_train = Data_train.loc[:,['tsLocal','load','load_ref','load-1w','load','rw','rd']+feature_names].dropna()
d_test = Data_test.loc[:,['tsLocal','load','load_ref','load-1w','load-1d','rw','rd']+feature_names].dropna()

# run individual models for each day of week
d_test['load_predict']= float('NaN')

feature_names = ['rw-1','rw-2','d','h']

X_train, y_train = d_train.loc[:,feature_names], d_train.loc[:,'rw']
X_test, y_test = d_test.loc[:,feature_names], d_test.loc[:,'rw']
mod = SVR(C=C, epsilon=epsilon).fit(X_train,y_train)
d_test.loc[:,'load_predict_w_1'] = (mod.predict(X_test))*d_test.loc[:,'load-1w']
mod.fit(X_train, y_train)
d_test.loc[:,'load_predict_w_2'] = (mod.predict(X_test))*d_test.loc[:,'load-1w']

feature_names = ['rd-1','rd-2','d','h']

X_train, y_train = d_train.loc[:,feature_names], d_train.loc[:,'rd']
X_test, y_test = d_test.loc[:,feature_names], d_test.loc[:,'rd']
mod = SVR(C=C, epsilon=epsilon).fit(X_train,y_train)
d_test.loc[:,'load_predict_d_1'] = (mod.predict(X_test))*d_test.loc[:,'load-1d']
mod.fit(X_train, y_train)
d_test.loc[:,'load_predict_d_2'] = (mod.predict(X_test))*d_test.loc[:,'load-1d']
        
l = d_test['load']-0.25*(d_test['load_predict_d_1']+d_test['load_predict_w_1']+d_test['load_predict_d_2']+d_test['load_predict_w_2'])
[mean_set7,max_set7] = [np.mean(np.absolute(l)),np.max(np.absolute(l))]

print [mean_set7,max_set7]

[411.82212750981074, 5673.055868859723]
